In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

root_dir = '/content/gdrive/MyDrive/Data'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
import os
import random
import shutil
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import csv
import tensorflow as tf
from tensorflow import keras

In [8]:
from pandas.core.frame import DataFrame
from time import sleep

def PreprocessAWSFile(aws_file, sheet_name):
  aws = pd.read_excel(aws_file, sheet_name=9)
  aws = aws.drop(['Line', 'Epoch', 'Day', 'Seconds', 'Off-Wrist Status', 'Marker', 
                  'White Light', 'Red Light', 'Green Light', 'Blue Light', 
                  'Sleep/Wake', 'Mobility', 'Interval Status', 'S/W Status'], axis=1)

  aws_complete_minute = pd.DataFrame(aws['Activity'])
  aws_complete_minute['Label'] = aws.apply(
      lambda x: str(x['Date'])[:10] + ' ' + x['Time'].strftime("%H:%M:%S"), axis=1)

  aws_complete_half_minute = pd.DataFrame({'Activity': [np.nan] * aws.shape[0]})
  aws_complete_half_minute['Label'] = aws.apply(
      lambda x: str(x['Date'])[:10] + ' ' + x['Time'].strftime("%H:%M")+':30', axis=1)

  aws = pd.concat([aws_complete_half_minute, aws_complete_minute]).sort_values(
      by=['Label']).reset_index(drop=True)

  aws['Activity'] = aws['Activity'].interpolate().replace(np.nan, 0)
  return aws

def PreprocessAX3File(ax3_file, chunks = 2):
  ax3 = pd.read_csv(ax3_file, header=None, squeeze=True)
  section_size = int(ax3.shape[0] / chunks)
  for i in range(chunks):
    ax3_processed_part = PreprocessAX3Data(ax3[i * section_size: section_size * (i + 1)])
    ax3_processed_part.to_pickle('part' + str(i+1) + '.pkl')
    ax3_processed_part = None
    print(i+1)
    sleep(40) # Waiting for RAM to get emptied
  ax3 = None

def GetProcessedAX3Data(chunks = 2):
  ax3 = None
  for i in range(chunks):
    file_name = 'part' + str(i+1) + '.pkl'
    temp = pd.read_csv(file_name)
    ax3 = pd.concat([ax3, temp])
    os.remove(file_name)
    temp = 0
  ax3 = ax3.reset_index()
  return ax3

def PreprocessAX3Data(ax3):
  ax3.columns = (['Date_time', 'X', 'Y', 'Z', 'Light', 'Temp'])
  ax3 = ax3.drop(["Light"], axis=1)
  ax3 = ax3[:-1]

  ax3['Label'] = ax3['Date_time'].map(lambda x : 
                x[0:17] + str(int(float(x[17:])>=30)*3) + "0")

  ax3 = ax3.groupby(['Label']).agg({'X': lambda x : list(x), 
                                    'Y': lambda x : list(x), 
                                    'Z': lambda x : list(x), 
                                    'Temp': lambda x : list(x)}).reset_index()
  print(len(ax3.iloc[10,2]))

  return ax3

Preprocessing AWS Data

In [33]:
aws_file = os.path.join(root_dir, "18 participants/AWS/SDRI001_AWS_ALL_ALLV4_N1.xlsx")
aws = PreprocessAWSFile(aws_file, 0)
print(aws.head(5))

   Activity                Label
0       0.0  2020-02-17 12:54:00
1       0.0  2020-02-17 12:54:30
2       0.0  2020-02-17 12:55:00
3       0.0  2020-02-17 12:55:30
4       0.0  2020-02-17 12:56:00


Preprocessing AX3 Data

In [4]:
ax3_extracted_dir = os.path.join(root_dir, "18 participants/AX3/Extracted")
psg_dir = os.path.join(root_dir, "18 participants/PSG")

ax3_extracted_files = os.listdir(ax3_extracted_dir)
psg_files = os.listdir(psg_dir)

participant = 13

ax3_file = os.path.join(ax3_extracted_dir, 
                        "AX3_step1_sub" + str(participant) + ".csv")

PreprocessAX3File(ax3_file, 4)


3064
1
3052
2
3046
3
3028
4


Merging AX3 and AWS Data

In [7]:
ax3.to_csv("AX3_step1_sub17.csv", index=False)

In [6]:
ax3 = GetProcessedAX3Data(4)

print(ax3.columns)




Index(['index', 'Label', 'X', 'Y', 'Z', 'Temp'], dtype='object')


In [9]:
ax3_extracted_dir = os.path.join(root_dir, "18 participants/AX3/Extracted")

participant = 18

ax3_file = os.path.join(ax3_extracted_dir, 
                        "AX3_step1_sub" + str(participant) + ".csv")


ax3_2 = pd.read_csv(ax3_file, header=None, squeeze=True)
ax3_2.columns = (['Date_time', 'X', 'Y', 'Z', 'Light', 'Temp'])
ax3_2 = ax3_2.drop(["Light"], axis=1)
ax3_2 = ax3_2[:-1]

ax3_2['Label'] = ax3_2['Date_time'].map(lambda x : 
              x[0:17] + str(int(float(x[17:])>=30)*3) + "0")

ax3_2 = ax3_2.groupby(['Label']).agg({'X': lambda x : list(x), 
                                  'Y': lambda x : list(x), 
                                  'Z': lambda x : list(x), 
                                  'Temp': lambda x : list(x)}).reset_index()

print(ax3_2.head(5))

print(ax3_2.iloc[1,2])
print(len(ax3_2.iloc[1,2]))


                 Label                                                  X  \
0  2020-03-02 10:00:00  [-0.609375, -0.5, -0.53125, -0.28125, -0.53125...   
1  2020-03-02 10:00:30  [-0.046875, -0.046875, -0.03125, -0.03125, -0....   
2  2020-03-02 10:01:00  [-0.25, -0.25, -0.25, -0.25, -0.25, -0.265625,...   
3  2020-03-02 10:01:30  [-0.03125, -0.03125, -0.03125, -0.015625, -0.0...   
4  2020-03-02 10:02:00  [-0.03125, -0.015625, -0.015625, -0.015625, -0...   

                                                   Y  \
0  [-0.046875, 0.875, 0.75, 0.375, 0.90625, 0.718...   
1  [0.953125, 0.953125, 0.953125, 0.953125, 0.953...   
2  [0.921875, 0.921875, 0.9375, 0.921875, 0.92187...   
3  [0.953125, 0.953125, 0.953125, 0.953125, 0.953...   
4  [0.953125, 0.953125, 0.953125, 0.953125, 0.953...   

                                                   Z  \
0  [-0.4375, -0.125, -0.21875, 0.03125, -0.296875...   
1  [0.078125, 0.078125, 0.09375, 0.078125, 0.0781...   
2  [-0.046875, -0.046875, -0.046

In [7]:
ax3_3 = pd.read_pickle("part1.pkl")

ax3_3['']

print(ax3_3.head(5))
print(ax3_3.iloc[10,2])
print(len(ax3_3.iloc[10,2]))


                 Label                                                  X  \
0  2020-02-28 13:33:00  [-0.40625, -0.875, -0.78125, -0.625, -0.71875,...   
1  2020-02-28 13:33:30  [-0.046875, -0.046875, -0.046875, -0.046875, -...   
2  2020-02-28 13:34:00  [-0.046875, -0.046875, -0.046875, -0.046875, -...   
3  2020-02-28 13:34:30  [-0.046875, -0.046875, -0.046875, -0.046875, -...   
4  2020-02-28 13:35:00  [-0.046875, -0.046875, -0.046875, -0.046875, -...   

                                                   Y  \
0  [0.515625, -0.640625, -0.734375, -0.671875, -0...   
1  [-0.96875, -0.96875, -0.96875, -0.96875, -0.96...   
2  [-0.96875, -0.96875, -0.96875, -0.96875, -0.96...   
3  [-0.96875, -0.96875, -0.96875, -0.96875, -0.96...   
4  [-0.96875, -0.96875, -0.96875, -0.96875, -0.96...   

                                                   Z  \
0  [-0.875, 0.1875, 0.0625, 0.078125, 0.140625, 0...   
1  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
2  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0

In [16]:
ax3 = None
ax3_2 = None
ax3_3 = None
